# Spaceship titanic

In [1]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import gc

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn import datasets, linear_model
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

pd.set_option('display.max_columns', None)   

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# import eli5
# from eli5.sklearn import PermutationImportance

C:\Users\79263\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('C:/Users/79263/Downloads/spaceship-titanic/train.csv')

In [3]:
train.head()
train = train.drop(['Name', 'Destination', 'PassengerId'], axis=1)

In [4]:
train['HomePlanet'] = train['HomePlanet'].map({'Earth' : 0, 'Europa' : 1, 'Mars': 2})
train

,HomePlanet,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,False,B/0/P,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0.0,False,F/0/S,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,1.0,False,A/0/S,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,1.0,False,A/0/S,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0.0,False,F/1/S,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...
8688,1.0,False,A/98/P,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,0.0,True,G/1499/S,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,0.0,False,G/1500/S,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,1.0,False,E/608/S,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [5]:
train['CryoSleep'] = train['CryoSleep'].replace({'False': 0, 'True': 1}).fillna(0)
train['CryoSleep'] = train['CryoSleep'].astype(float)
train['VIP'] = train['VIP'].replace({'False': 0, 'True': 1}).fillna(0)
train['VIP'] = train['VIP'].astype(float)
train['Transported'] = train['Transported'].replace({'False': 0, 'True': 1})
train['Transported'] = train['Transported'].astype(float)

In [6]:
train

,HomePlanet,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,F/0/S,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0
2,1.0,0.0,A/0/S,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0.0
3,1.0,0.0,A/0/S,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0.0
4,0.0,0.0,F/1/S,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
8688,1.0,0.0,A/98/P,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0.0
8689,0.0,1.0,G/1499/S,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8690,0.0,0.0,G/1500/S,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1.0
8691,1.0,0.0,E/608/S,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0.0


In [7]:
train = train.drop('Cabin', axis=1)
train['Age'] = train['Age'].fillna(train.Age.median())
train['RoomService'] = train['RoomService'].fillna(train.RoomService.median())
train['FoodCourt'] = train['FoodCourt'].fillna(train.FoodCourt.median())
train['ShoppingMall'] = train['ShoppingMall'].fillna(train.ShoppingMall.median())
train['Spa'] = train['Spa'].fillna(train.Spa.median())
train['VRDeck'] = train['VRDeck'].fillna(train.VRDeck.median())

In [8]:
train = train.fillna(method='ffill')

In [9]:
y = train['Transported']

In [10]:
x_train, x_holdout, y_train, y_holdout = train_test_split(train.drop('Transported', axis=1), y, test_size=0.3, random_state=17)

In [11]:
tree = DecisionTreeClassifier(max_depth=5, random_state=17)
tree.fit(x_train, y_train)
tree_pred = tree.predict(x_holdout)
accuracy_score(y_holdout, tree_pred) 

0.7795245398773006

In [12]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_holdout)
accuracy_score(y_holdout, knn_pred) 

0.7848926380368099

In [13]:
X, y = train.drop('Transported', axis=1), np.asarray(train["Transported"],dtype='int8')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

results = cross_val_score(rfc, X, y, cv=skf)

results.mean()

0.7862627556675497

In [14]:
parameters = {'n_estimators': [50,75,100], 'max_features': [3, 7, 13], 'min_samples_leaf': [3, 5, 7], 'max_depth': [5,10,15]}
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)
gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1)
gcv.fit(X, y)
print(gcv.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
{'max_depth': 15, 'max_features': 3, 'min_samples_leaf': 7, 'n_estimators': 100}


In [15]:
X, y = train.drop('Transported', axis=1), np.asarray(train["Transported"],dtype='int8')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True, max_depth=15, max_features=3, min_samples_leaf=7, n_estimators=100)

results = cross_val_score(rfc, X, y, cv=skf)

results.mean()

0.7966166421054651

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Conv2D, Input,InputLayer
from keras.utils import np_utils
from keras.models import Sequential,Model
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

import keras
from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Sequential,Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras import backend as K

In [17]:
y_train = train['Transported'].astype('float32')
x_train = train.drop(['Transported'], axis=1).astype('int32')

In [18]:
x_train = x_train.values.reshape(-1, 3, 3, 1)

In [19]:
x_train = x_train/255

In [20]:
y_train = np_utils.to_categorical(y_train, 2)

In [21]:
x_train, x_cv, y_train, y_cv = train_test_split(x_train, y_train, test_size=0.1, random_state=17)

In [22]:
input_shape = (3,3,1)
X_input = Input(input_shape)

x = Conv2D(32,(3, 3), activation = 'relu', padding='same')(X_input)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)

model = Model(inputs=X_input, outputs=x, name='Predict')
model.summary()

Model: "Predict"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 3, 1)]         0         
                                                                 
 conv2d (Conv2D)             (None, 3, 3, 32)          320       
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 32)          9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 1, 32)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1, 1, 32)          0         
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                           

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(x_train, y_train, epochs=60, batch_size=32)

Epoch 1/60
245/245 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7603
Epoch 2/60
245/245 [==============================] - 0s 2ms/step - loss: 0.5050 - accuracy: 0.7832
Epoch 3/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4931 - accuracy: 0.7886
Epoch 4/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4933 - accuracy: 0.7856
Epoch 5/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4885 - accuracy: 0.7863
Epoch 6/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4880 - accuracy: 0.7873
Epoch 7/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4877 - accuracy: 0.7883
Epoch 8/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4862 - accuracy: 0.7901
Epoch 9/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4833 - accuracy: 0.7909
Epoch 10/60
245/245 [==============================] - 0s 2ms/step - loss: 0.4815 - accuracy: 0.7893

In [25]:
sgd = SGD(learning_rate=0.00001, momentum=0.001, decay=0.0, nesterov=False) 
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1)

245/245 [==============================] - 1s 2ms/step - loss: 0.4362 - accuracy: 0.7895


In [26]:
test = pd.read_csv('C:/Users/79263/Downloads/spaceship-titanic/test.csv')
test['HomePlanet'] = test['HomePlanet'].replace({'Earth' : 0, 'Europa' : 1, 'Mars': 2}).fillna(0)
test['CryoSleep'] = test['CryoSleep'].replace({'False': 0, 'True': 1}).fillna(0)
test['CryoSleep'] = test['CryoSleep'].astype(float)
test['VIP'] = test['VIP'].replace({'False': 0, 'True': 1}).fillna(0)
test['VIP'] = test['VIP'].astype(float)

test = test.drop(['Cabin', 'Name', 'Destination', 'PassengerId'], axis=1)
test['Age'] = test['Age'].fillna(test.Age.median())
test['RoomService'] = test['RoomService'].fillna(test.RoomService.median())
test['FoodCourt'] = test['FoodCourt'].fillna(test.FoodCourt.median())
test['ShoppingMall'] = test['ShoppingMall'].fillna(test.ShoppingMall.median())
test['Spa'] = test['Spa'].fillna(test.Spa.median())
test['VRDeck'] = test['VRDeck'].fillna(test.VRDeck.median())
test = test.fillna(method='ffill')

test

,HomePlanet,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0
2,1.0,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0
4,0.0,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
4272,0.0,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0
4273,0.0,0.0,42.0,0.0,0.0,847.0,17.0,10.0,144.0
4274,2.0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0
4275,1.0,0.0,26.0,0.0,0.0,2680.0,0.0,0.0,523.0


In [27]:
x_test = test.astype('float32')
x_test = x_test/255
x_test = x_test.values.reshape(-1,3,3,1)
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)

134/134 [==============================] - 0s 857us/step


In [28]:
test.insert(1, 'Transported', y_pred, allow_duplicates = False)

In [29]:
test

,HomePlanet,Transported,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,1,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0
2,1.0,1,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0
4,0.0,1,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4272,0.0,1,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0
4273,0.0,0,0.0,42.0,0.0,0.0,847.0,17.0,10.0,144.0
4274,2.0,1,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0
4275,1.0,1,0.0,26.0,0.0,0.0,2680.0,0.0,0.0,523.0


In [30]:
test['Transported'] = test['Transported'].replace({1 : 'True', 0 : 'False'})

In [31]:
test

,HomePlanet,Transported,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,True,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,False,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0
2,1.0,True,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,True,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0
4,0.0,True,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4272,0.0,True,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0
4273,0.0,False,0.0,42.0,0.0,0.0,847.0,17.0,10.0,144.0
4274,2.0,True,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0
4275,1.0,True,0.0,26.0,0.0,0.0,2680.0,0.0,0.0,523.0


In [32]:
my_submission = pd.DataFrame({'PassengerId': list(range(1, len(y_pred)+1)), 'Transported': test['Transported']})
my_submission.to_csv('C:/Users/79263/Downloads/www_rev', index=False)